# **Ensemble Learning Using Stacking**

We will implement stacking ensemble learning with VGG16 and ResNet50 as base models. We will train these models separately and then use their predictions as features for a meta-model (e.g., a logistic regression classifier) to make the final prediction.

#### Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#### Data Preparation

In [ ]:
# Import dataset from kaggle
!kaggle datasets download -d smaranjitghose/corn-or-maize-leaf-disease-dataset

In [ ]:
# Unzip the imported dataset
!unzip /content/corn-or-maize-leaf-disease-dataset.zip -d /content

In [ ]:
data_dir = '/content/data'
batch_size = 32
target_size = (224, 224)

# Define ImageDataGenerators for training and validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_dataset = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=False
)

val_dataset = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

#### Define and Train Base Models

Define VGG16 and ResNet50 models with transfer learning and train them.

In [ ]:
def create_base_model(base_model_class, input_shape, num_classes):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
input_shape = (224, 224, 3)
num_classes = len(train_dataset.class_indices)

In [ ]:
# Define models
vgg16_model = create_base_model(VGG16, input_shape, num_classes)
resnet50_model = create_base_model(ResNet50, input_shape, num_classes)

In [ ]:
# Compile and train VGG16 model
vgg16_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg16_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

In [ ]:
# Compile and train ResNet50 model
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet50_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

#### Extract Features for Stacking

Extract features from the base models and prepare the data for the meta-model.

In [ ]:
def extract_features(model, dataset):
    features = []
    labels = []
    for data_batch, label_batch in dataset:
        features_batch = model.predict(data_batch)
        features.append(features_batch)
        labels.append(label_batch)
        if len(features) * dataset.batch_size >= dataset.samples:
            break
    return np.vstack(features), np.concatenate(labels)

In [ ]:
# Extract features
vgg16_features, vgg16_labels = extract_features(vgg16_model, train_dataset)
resnet50_features, resnet50_labels = extract_features(resnet50_model, train_dataset)

# Stack features
X_train_stacked = np.hstack((vgg16_features, resnet50_features))

In [ ]:
# Extract validation features
vgg16_val_features, vgg16_val_labels = extract_features(vgg16_model, val_dataset)
resnet50_val_features, resnet50_val_labels = extract_features(resnet50_model, val_dataset)

# Stack validation features
X_val_stacked = np.hstack((vgg16_val_features, resnet50_val_features))

In [ ]:
# Ensure labels match
assert np.array_equal(vgg16_labels, resnet50_labels)
assert np.array_equal(vgg16_val_labels, resnet50_val_labels)

#### Train Meta-Model
Train a logistic regression classifier as the meta-model.

In [ ]:
# Train-test split for meta-model
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(
    X_train_stacked, vgg16_labels, test_size=0.2, random_state=42
)

In [ ]:
# Define and train meta-model
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(X_train_meta, y_train_meta)

In [ ]:
# Evaluate meta-model
y_pred_meta = meta_model.predict(X_test_meta)
print(f'Logistic Regression Meta-Model Accuracy: {accuracy_score(y_test_meta, y_pred_meta)}')